In [1]:
import networkx as nx
import pandas as pd

In [2]:
f = open("data/id_symbol.txt", "r")

id_to_symbol = {}

for line in f.readlines():
  id, symbol = line.split(" ")
  id_to_symbol[id] = symbol.strip()

f.close()

In [3]:
disease_protein_df = pd.read_csv("data/disease_protein.csv", sep=":")

disease_proteins = []

for id in disease_protein_df.iloc[15]["proteins"].split("/"):
  disease_proteins.append(id_to_symbol[id])

In [4]:
G = nx.read_edgelist("data/ppin.txt",comments="#",nodetype=str)

In [5]:
GC = G.subgraph(max(nx.connected_components(G), key=len)).copy()

In [6]:
seeds = []

for prot in disease_proteins:
  if prot in GC:
    seeds.append(prot)

print(len(seeds))

40


In [24]:
import utils

In [176]:
N = GC.number_of_nodes()
s0 = len(seeds)

p = {}

for node in GC.nodes():

  # Find number of links to seeds
  ks = 0
  for neighbour in GC.neighbors(node):
    if neighbour in seeds:
      ks += 1
  
  p[node] = utils.pvalue(ks, GC.degree(node), N, s0)

In [177]:
import pcst_fast

In [286]:
node_id = {node: i for i, node in enumerate(GC.nodes)}
id_node = {v: k for k, v in node_id.items()}
edge_id = [(node_id[u], node_id[v]) for u, v in GC.edges]
prizes = []
for node in GC.nodes():
  if node in seeds:
    prizes.append(10)
  else:
    prizes.append(0.1)
edge_cost = [1 for i in range(GC.number_of_edges())]

In [287]:
counts = {}
for seed in seeds:
  root = node_id[seed]
  vertices, edges = pcst_fast.pcst_fast(edge_id, prizes, edge_cost, root, 1, "gw", 0)
  for v in vertices:
    if v in counts:
      counts[v] += 1
    else:
      counts[v] = 1

In [288]:
t = 0.95

final = [id_node[node] for node, count in counts.items() if count/len(seeds) >= t]
print(len(final))

45


In [289]:
for node in final:
  if node not in seeds:
    print(p[node])

0.2646349919574831
0.12077998250332189
0.00013428870117501867
0.004637497792813714
0.0001827178115384471
